# Банки — Анализ оттока клиентов

<big><b>Задача проекта</b></big>  
Проанализировать клиентов регионального банка и выделить портрет клиентов, которые склонны уходить из банка.

<big><b>Описание датасета</b></big>  
Датасет содержит данные о клиентах банка «Метанпром». Банк располагается в Ярославле и областных городах: Ростов Великий и Рыбинск.

<big><b>Колонки:</b></big>

- `user_id` — идентификатор пользователя,
- `score` — баллы кредитного скоринга,
- `city` — город,
- `gender` — пол,
- `age` — возраст,
- `equity` — количество объектов в собственности,
- `balance` — баланс на счёте,
- `products` — количество продуктов, которыми пользуется клиент,
- `credit_сard` — есть ли кредитная карта,
- `last_activity` — активный клиент,
- `est_salary` — заработная плата клиента,
- `churn` — ушёл или нет.

##### Материалы:
Презентация
https://disk.yandex.ru/i/fGzp-W_-JZl3zA


## 1. Открываем файл с данными и изучаем общую информацию. 

In [ ]:
# загружаем необходимые библиотеки
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go

from scipy import stats as st

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score


In [ ]:
# загружаем данные из файла в датафрейм
path = "https://drive.google.com/uc?export=download&id=1-U61mhTz_N1ARjy2XSAZ7IlQqGjeqP0F"
df = pd.read_csv(path)

In [ ]:
# выводим первые пять строк на экран
df.head(5)

In [ ]:
# выводим основную информацию о датафрейме с помощью метода info()
df.info()

Вывод: вывели датафрейм, затем методом info() запросили информацию из всех атрибутов таблицы. Изучили результаты и будем выбирать тактику работы с данными. Мы видим, что необхоимо будет поменять название столбцов(привести к нижнему регистру),заполнить пропущенные данные в balance.

## 2. Предобработка данных

### 2.1 Изменение названия столбцов

In [ ]:
# заменим названия столбцов (приведем к нижнему регистру)
df.columns = df.columns.str.lower()

In [ ]:
# поменяем название столбца 'userid'
df.rename(columns={'userid': 'user_id'},inplace = True)

### 2.2 Проверка пропущенных значений

In [ ]:
# выведем количество пропущенных значений для каждого столбца
df.isna().sum()

Всего пропусков 2295 и все они в столбце 'balance'.

In [ ]:
### КОД РЕВЬЮЕРА
import numpy as np

df.balance.hist(bins=np.arange(0,5e6,5e4)).set(title="Распределение до импутации");

Если посмотрим на гистограмму распределения остатков на счете, то можно сделать вывод, что распределение почти нормальное. Посмотрим на распределение пропусков баланса в отточной группе и в неотточной.

In [ ]:
df.groupby('churn')['balance'].apply(lambda x: x.isna().sum())


Получается, что пропуски приходятся в основном на тех клиентов, которые остались. Посмотрим на связь пропусков с кредитными картами.

In [ ]:
# выведем количество клиентов с кредитными картами и без
df['credit_card'].value_counts()

In [ ]:
# сгруппируем по наличию кредитной карты и посмотрим на пропуски баланса
df.groupby('credit_card')['balance'].apply(lambda x: x.isna().sum())

Пропуски в 4 раза чаще встречаются для клиентов с кредитной картой. Это может говорить о том, что у них только кредитные счета, а дебетовых карт у них нет.

In [ ]:
df.query('balance == 0')

Нулевых остатков в датасете всего лишь 2 ед.

In [ ]:
# посмотрим как пропуски распределены по продуктам, которыми пользуется клиент
df.groupby('products')['balance'].apply(lambda x: x.isna().sum())

Большое количество значений столбца balance пропущено у пользователей с одним продуктом.

##### Вывод:  
Наибольшее количество пропусков получилось у неотточных клиентов, владеющие кредитными картами, и пользующиеся в основном одним продуктом.
Можем предположить,что речь идет о клиентах, использующих только кредитные продукты и не держащих собственных средств на дебетовом счете. В таком случае у клиента как раз может быть один продукт: например, кредитка. Таким образом, мы выяснили, что значения в столбце balance отсутствуют не случайно, и отбрасывание или заполнение пропусков приведёт к заметным искажениям.

In [ ]:
### КОД РЕВЬЮЕРА
import numpy as np

df.balance.hist(bins=np.arange(0,5e6,5e4)).set(title="Распределение после импутации");

In [ ]:
### КОД РЕВЬЮЕРА
nan_bal_cc_groups = df.balance.isna().map({False:"have balance ",True:"no balance "})\
                  + df.credit_card.map({1:"and credit card",0:"and no credit card"})

sns.kdeplot(data=df,hue=nan_bal_cc_groups,x='score',common_norm=False);

### 2.3 Обработка дубликатов

In [ ]:
# выведем на экран количество строк-дубликатов в данных
df.duplicated().sum()

Учитывая, что количество пропусков очень маленькое строки с отсутствующими данными можно удалить.

In [ ]:
# удалим дубликаты
df = df.drop_duplicates()

In [ ]:
### КОД РЕВЬЮЕРА
USERID = 'user_id'
df_userid_dups = df[df.duplicated(keep='first',subset=USERID)]\
                    .merge(df[df.duplicated(keep='last',subset=USERID)],on=USERID) 
df_userid_dups[df_userid_dups.columns.sort_values()].sort_values(['age_x','balance_x'])

Вывод: подготовили данные для дальнейшего исследовательского анализа - названия столбцов привели в соответствующий вид, заменили пропущенные значения в столбце balance и удалили дубликаты.

## 3. Исследовательский анализ данных

In [ ]:
# выведем средние значения показателей
df.describe()

In [ ]:
# выведем на график соотношение ушедших клиентов и оставшихся в процентном измерении
fig = go.Figure(data=go.Pie(labels=df['churn'].value_counts().reset_index()['index'], values=df['churn'].value_counts()))
fig.update_layout(
    title={
        'text':'Отток клиентов в процентном соотношении',
        'y':0.9,
        'x':0.7})
for trace in fig.data:
    trace['labels']=['Остался', 'Ушел']
fig.show()

За месяц ушло 18.2% клиентов.

### 3.1 Распределение каждой величины в разрезе оттока

In [ ]:
# визуализируем факт оттока в зависимости от баллов кредитного скоринга
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='score', stat='density', common_norm=False, palette='flag')
plt.title('Распределение баллов кредитного скоринга ')
plt.xlabel('Баллы')
plt.ylabel('Плотность распределения')
plt.show()

К наиболее отточным относятся клиенты с 825-900 баллами кредитного скоринга.

In [ ]:
# визуализируем факт оттока в зависимости от городов
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='city', stat='density', common_norm=False, palette='flag')
plt.title('Отточность клиентов в разрезе городов')
plt.xlabel('Города')
plt.ylabel('Плотность распределения')
plt.show()

Из графика видно, что город Рыбинск менее отточный, а Ярославль более отточный.

In [ ]:
# визуализируем факт оттока в зависимости от пола клиента
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='gender', stat='density', common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от пола')
plt.xlabel('Пол клиента')
plt.ylabel('Плотность распределения')
plt.show()

Мужчины более склоны к уходу, чем женщины.

In [ ]:
# визуализируем факт оттока в зависимости от возраста клиента
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='age', stat='density', common_norm=False, palette='flag',kde=True)
plt.title('Отточность клиентов в зависимости от возраста')
plt.xlabel('Возраст клиентов')
plt.ylabel('Плотность распределения')
plt.show()

Из графика видно, что более склоные к уходу клиенты имеют возраст 25-35 и 50-60 лет.

In [ ]:
# визуализируем факт оттока в зависимости от количества объектов в собственности
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='equity', stat='density', common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от количества объектов в собственности')
plt.xlabel('Количество объектов')
plt.ylabel('Плотность распределения')
plt.show()

Чаще всего уходят клиенты с 4 и более объектами в собственности. С одним и двумя объектами клиенты реже попадают в отток.

In [ ]:
max(df['balance'])

In [ ]:
# визуализируем факт оттока в зависимости от баланса на счете
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='balance',bins=600,stat='density', common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от баланса')
plt.xlabel('Баланс')
plt.ylabel('Плотность распределения')
plt.xlim([0, 10000000])
plt.show()

Более склоные к уходу клиенты - это клиенты у которых баланс на счете составляет от 100 т.р до 400 т.р.

In [ ]:
### КОД РЕВЬЮЕРА
df_temp = df.assign(balance = df.balance.clip(0,6e6))
ax = sns.kdeplot(data = df_temp, x = 'balance', hue = 'churn', common_norm=False)
ax.set(title=f"Распределения отточных и неотточных по балансу")
ax2 = ax.twinx()
df_temp.assign(balance = df_temp.balance.round(-5)).groupby('balance')\
       .agg({'churn':'mean'}).rolling(5,center=True).mean().plot(ax=ax2,color="violet",ls=":",lw=4);
ax.grid(False,'both','both');

In [ ]:
# визуализируем факт оттока в зависимости от количества продуктов, которыми пользуется клиент
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='products',stat='density', common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от количества продуктов')
plt.xlabel('Количество продуктов')
plt.ylabel('Плотность распределения')
plt.show()

Наиболее отточные клиенты пользуются 4 продуктами.

In [ ]:
# заменим значения 0 и 1 на "нет", "да"
df['credit_card']=df['credit_card'].replace(1,"есть")
df['credit_card']=df['credit_card'].replace(0,"нет")

In [ ]:
# визуализируем факт оттока в зависимости от наличия кредитной карты
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='credit_card', stat='density', bins=2, common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от наличия кредитной карты')
plt.xlabel('Наличие кредитной карты')
plt.ylabel('Плотность распределения')
plt.show()

Наиболее отточные клиенты - это клиенты, у которых нет кредитной карты.

In [ ]:
# заменим значения 0 и 1 на "нет", "да"
df['last_activity']=df['last_activity'].replace(1,"да")
df['last_activity']=df['last_activity'].replace(0,"нет")

In [ ]:
# визуализируем факт оттока в зависимости от активности клиента
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='last_activity', stat='density', common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от активности клиента')
plt.xlabel('Активность клиента')
plt.ylabel('Плотность распределения')
plt.show()

Активные пользователи являются более отточными.

In [ ]:
# визуализируем факт оттока в зависимости от заработной платы клиента
fig,ax = plt.subplots(1,sharey=False,figsize=(10,7))
sns.histplot(data=df, hue='churn', x='est_salary', stat='density',bins=50, common_norm=False, palette='flag')
plt.title('Отточность клиентов в зависимости от заработной платы')
plt.xlabel('Размер заработной платы')
plt.ylabel('Плотность распределения')
plt.show()

Клиенты с доходом от 100 тр. до 200 т.р являются более отточными , с доходом до 100 т.р. менее отточными.

Посмотрим на проценты оттока в каждой категории

In [ ]:
# заменим названия столбцов на более удобные
columns = ['id пользователя', 'кредитный скоринг', 'город', 'пол', 'возраст', 'объекты в собственности',
              'остаток на счете', 'используемые продукты', 'кредитная карта', 'лояльность', 'зарплата',
              'ушел/остался']
df_columns = dict(zip(df.columns, columns))


In [ ]:
# удалим 'user_id' и сохраним данные во временную переменную df_no_id
df_no_id = df.drop('user_id', axis=1)

In [ ]:
# отфильтруем данные, оставим клиентов, которые ушли, и сохраним в переменной df_ch
df_ch = df.query('churn == 1')

In [ ]:
# выделим категориальные данные
df_category = df_no_id[['city', 'gender', 'equity', 'products', 'credit_card', 'last_activity', 'churn']]

In [ ]:
# определим функции для работы в этом разделе
def ch (cols):
    
    df_all = df[cols].value_counts().sort_index()
    df_churn = df_ch[cols].value_counts().sort_index()
    percentage = (df_churn / df_all).sort_index()
    print(percentage)
    
    churn_0 = df_all.plot(kind='bar', color='grey', figsize = (12,8))
    churn_1 = df_churn.plot(kind='bar', color='blue',figsize = (12,8))
  
    
    # выведем процент оттока в каждом классе
    for i, rect in enumerate(churn_1.patches):
        n = int(len(churn_1.patches) / 2.0) # выведем только вторую часть
        if i >= n:
            height = rect.get_height()

            plt.text(rect.get_x() + rect.get_width() / 2.0, height / 4.0, f'{percentage.iloc[int(i-n)]:1.1%}', 
                     ha='center', va='bottom', color='black')

    plt.title(f'Распределение признака "{df_columns[cols]}"')
    plt.xlabel('')
    plt.xticks(rotation=360)
    plt.ylabel('Количество клиентов')
    plt.grid(axis='y', linestyle='--')
    plt.legend(['остался', 'ушел'])
    

    

with sns.axes_style('white'):
    for cols in df_category.drop('churn', axis=1).columns:
        ch(cols)
        plt.show()

По графикам оттока среди категориальных величин, мы видим, что самые большие расхождения по признаку loyalty(активный) - среди отточных их 24.5%, а среди оставшихся в два раза меньше - 11.4%. Так же сильно отличается признак gender(пол) - среди мужчин отточных 23.7%, а среди женщин в два раза меньше - 12.8%. 

### 3.2 Портреты клиентов

Посмотрим на средние значения характеристик в двух группах - тех, кто ушел и тех, кто остался.

In [ ]:
df_no_id['gender'] = df_no_id['gender'].replace("М", 1)
df_no_id['gender'] = df_no_id['gender'].replace("Ж", 0)
df_no_id['last_activity']=df_no_id['last_activity'].replace("да",1)
df_no_id['last_activity']=df_no_id['last_activity'].replace("нет",0)
df_no_id['credit_card']=df_no_id['credit_card'].replace("есть",1)
df_no_id['credit_card']=df_no_id['credit_card'].replace("нет",0)

In [ ]:
df_describe = df_no_id.agg(['mean', 'std']).T.round(2)
df_describe

In [ ]:
df_churn = df_no_id.groupby('churn').agg('mean').round(2).T
mean_churn = df_churn.join(df_describe)
mean_churn = mean_churn.rename(columns = {0:'mean_no_churn', 1:'mean_churn'})
mean_churn

Вывод на основе средних значений между характеристиками в разрезе отточные и неотточные клиенты:  
наибольшие расхождения видим по полу, по количеству объектов и продуктов, по лояльности, ну и по балансу.

Вывод:                                                                                            
Исходя из выведенных в п.3.1 и 3.2 графиков и подсчетов определим портрет клиента, склонного к оттоку.  
• клиенты – 25-35, 50-60 лет                                                                        
• мужчины                                                                                           
• не имеют кредитные карты                                                                         
• являются активными клиентами                                                                      
• основные города в оттоке – Ярославль и Ростов Великий                                             
• в среднем 5 объектов в собственности                                                              
• в среднем имеют более 1 оформленного продукта                                                     


In [ ]:
### КОД РЕВЬЮЕРА
data = pd.read_csv(path)
data.columns = data.columns.str.lower()
df_mean = data.join(pd.get_dummies(df.gender, drop_first = True)).groupby('churn')\
            .agg({col:("median" if col in ['est_salary','balance'] else "mean") 
                     for col in data.columns.union(['М']) 
                     if col not in ["user_id","churn",'city',"gender"]})\
            .T
df_mean['∂'] = df_mean[1] / df_mean[0] - 1
df_mean.sort_values('∂').style.format("{:_.2f}").background_gradient(subset="∂",cmap="vlag",vmin=-1, vmax=1)

### 3.3 Построим матрицу корреляции

In [ ]:
# заменим значения 'М' и 'Ж' на 1 и 0
df['gender'] = df['gender'].replace("М", 1)
df['gender'] = df['gender'].replace("Ж", 0)
df['gender'] = df['gender'].astype('int')

In [ ]:
# заменим значения "нет", "да" на 0 и 1
df['last_activity']=df['last_activity'].replace("да",1)
df['last_activity']=df['last_activity'].replace("нет",0)
df['last_activity'] = df['last_activity'].astype('int')

In [ ]:
# заменим значения "нет", "есть" на 0 и 1
df['credit_card']=df['credit_card'].replace("есть",1)
df['credit_card']=df['credit_card'].replace("нет",0)

In [ ]:
df

In [ ]:
# закодируем города в столбце city
df_new = pd.get_dummies(df, columns=['city'])
df_new

In [ ]:
# удалим столбец 'user_id'
df_corr = df_new.drop(['user_id'], axis=1).corr()[['churn']]
# отобразим heatmap
plt.figure(figsize = (15,15))
sns.heatmap(df_corr, square=True, vmax =0.5, annot = True)

plt.show()



С целевой переменной churn больше всего коррелирует products (количество продуктов, которыми пользуется клиент), equity (количество объектов в собственности), last_activity(активный клиент) и gender(пол).

## 4. Проверка статистических гипотез

Проверим равенство средних двух генеральных совокупностей - это пользователи, которые ушли - 1821, и пользователи, которые остались - 8165.                                                               

### 4.1 Гипотеза о различии дохода между отточными и неотточными клиентами

Проверим гипотезу различия дохода между теми клиентами, которые ушли и теми, которые остались.    
Нулевая: различия в доходах между теми клиентами, которые ушли и теми, которые остались нет.     
Альтернативная: существуют различия в доходах между теми клиентами, которые ушли и теми, которые остались.

In [ ]:
est_salary_churn = df.query('churn == 1').pivot_table(index = 'user_id', values = 'est_salary')
est_salary_no_churn = df.query('churn == 0').pivot_table(index = 'user_id', values = 'est_salary')
alpha = .05 # критический уровень статистической значимости
# если p-value окажется меньше него - отвергнем гипотезу

In [ ]:

alpha = .05 # критический уровень статистической значимости
results_est_salary = st.ttest_ind(est_salary_churn.values, est_salary_no_churn.values)
print('p-значение:', results_est_salary.pvalue)

if results_est_salary.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print(
        'Нулевую гипотезу не отклоняем. Нет оснований говорить о различии в среднем доходе между клиентами, которые ушли и которые остались.'
    ) 

Проверим гипотезу еще при помощи теста Манна-Уитни, так как наши данные подходят под условия применения данного теста.  
Наши гипотезы:  
Нулевая: различия в доходах между теми клиентами, которые ушли и теми, которые остались нет.  
Альтернативная: существуют различия в доходах между теми клиентами, которые ушли и теми, которые остались.

In [ ]:
alpha = 0.05 # критический уровень статистической значимости
results = st.mannwhitneyu(est_salary_churn['est_salary'], est_salary_no_churn['est_salary'])
print('p-значение: ', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print(
        'Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя'
    ) 

Гипотеза о том, что доход между отточными и неотточными клиентами отличается, подтвердилась.

Итак, мы получили расхождения в результатах t-теста и u-теста. Так как в столбце 'est_salary' мы наблюдаем аномально высокие значения, результаты параметрического t-теста могут исказиться. А непараметрический критерий Манна-Уитни устойчив к такого рода выбросам, поэтому в данном случае имеет смысл ориентироваться на него.

### 4.2 Гипотеза о  влияния возраста на отток

Проверим гипотезу влияния возраста на отток.                                                      
Нулевая: средний возраст в группе с оттоком и без одинаковый.                                         
Альтернативная: существуют различия в возрасте между группами с оттоком и без.

In [ ]:
age_churn = df.query('churn == 1').pivot_table(index = 'user_id', values = 'age')
age_no_churn = df.query('churn == 0').pivot_table(index = 'user_id', values = 'age')                                

In [ ]:
alpha = 0.05 # критический уровень статистической значимости
results_age = st.ttest_ind(age_churn.values,age_no_churn.values)

print('p-значение:', results_age.pvalue)

if results_age.pvalue < alpha:
    print('Отвергаем нулевую гипотезу: разница статистически значима')
else:
    print('Не получилось отвергнуть нулевую гипотезу, вывод о различии сделать нельзя')


Гипотеза подтвердила, что существуют различия в возрасте между группами с оттоком и без.

## 5. Вывод о влиянии стратегических показателей на отток

После исследовательского анализа данных мы можем выделить показатели, которые сильнее всего влияют на отток.  
Это наличие кредитной карты, возраст, пол (М), количество продуктов и количество объектов в собствености, а так же лояльность(0).  
Отток в Ростове Великом (18.8%) и в Ярославле (19%) оказались чуть больше, чем в Рыбинске (16.2%).
К оттоку оказались наиболее склонны лояльные молодые люди возраста 25-35 лет и мужчины старше 50, проживающие в Ростове Великом и Ярославле, пользующиеся четырьмя продуктами банка, владеющие 4 и более объектами собственности, зарабатывающие от 100 до 200 тыс.  
Наоборот, чаще всего остаются с банком нелояльные женщины моложе 50, проживающие в Рыбинске, пользующиеся 1-2 продуктами банка, владеющие 2-3 объектами собственности, зарабатывающие до 150 тыс. или от 200 тыс. и более.
Гипотеза о статистически значимом различии дохода клиентов, которые ушли, и тех, которые остались, подтвердилась.
Гипотеза о различии возраста тех, кто ушел в отток, и тех, кто остался тоже подтвердилась. 

## Сегментация по оттоку

<b> Выделим сегменты исходя из выводов о влиянии стратегических показателей на отток (п.5.5). </b>  
-посчитаем долю отточных клиентов мужского пола, не имеющих кредитные карты,  
-посчитаем долю отточных клиентов мужского пола с 4-мя продуктами,  
-посчитаем сколько отточных клиентов активных и имеющих более 4 объектов в собственности,  
-посчитаем долю зрелых отточных клиентов с балансом 100-400 т.р.,  
-посчитаем долю зрелых отточных клиентов с доходом 100-200 т.р.
-посчитаем отточных клиентов с баллами от 800 до 900 и балансом на счете от 100 до 400 т.р.,  


In [ ]:
# посчитаем сколько отточных клиентов среди мужчин без кредитной карты
df_pivot = df.pivot_table(
    index='gender', columns='credit_card', values='churn', aggfunc='mean'
)

sns.set(style='white')
plt.figure(figsize=(13, 9))
plt.title('% оттока клиентов в зависимости от пола и наличия кредитных карт')
sns.heatmap(df_pivot, annot=True, fmt='.2%', linewidths=1, linecolor='gray')
plt.xlabel('Кредитные карты') 
plt.ylabel('Пол'); 

Из графика видим, что самыми отточными клиентами в зависимости от пола и наличия кредитных карт являются мужчины, их уровень составляет - 35.07%.

In [ ]:
 # посчитаем сколько отточных мужчин с 4-мя продуктами
df_pivot_gender = df.pivot_table(
    index='gender', columns='products', values='churn', aggfunc='mean'
)
df_pivot_gender

Уровень оттока мужчин с 4-мя продуктами получается 67%

In [ ]:
# посчитаем сколько отточных клиентов активных и имеющих более 4 объектов в собственности
df_pivot_act = df.pivot_table(
    index='last_activity', columns='equity', values='churn', aggfunc='mean'
)
df_pivot_act

Видим, что активные клиенты, имеющие 4 и более объектов в собственности, отточнее всего. Активный клиент отточнее всего с 9-ю продуктами.

In [ ]:
# посчитаем долю зрелых отточных клиентов с балансом 100-400 т.р. на счете
df_age_balance = df.query('(60 > age > 50)&(400000 > balance > 100000)').agg({'churn':'mean'})
df_age_balance

Зрелые отточные клиенты с балансом 100-400 т.р. на счете составляют 26%

In [ ]:
# посчитаем долю зрелых отточных клиентов с доходом 100-200 т.р.
df_age_balance = df.query('(60 > age > 50)&(200000 > est_salary > 100000)').agg({'churn':'mean'})
df_age_balance

Зрелые отточные клиенты с доходом 100-200 т.р. на счете составляют 29%

In [ ]:
# посчитаем отточных клиентов с баллами от 800 до 900 и балансом на счете от 100 до 400 т.р.
df_score_balance = df.query('(900 > score > 800)&(400000 > balance > 100000)').agg({'churn':'mean'})
df_score_balance

Уровень отточных клиентов с баллами от 800 до 900 и балансом на счете от 100 до 400 т.р. составляют 19%. Этот показатель не очень высокий, поэтому не будем брать в счет при рекомендациях.

## Выводы и рекомендации по сегментам

#### Выводы: 
 
Портрет клиента, склонного к оттоку в целом таков:  
-доход от 100 т.р. до 200 т.р.  
-возраст - либо 25-25, либо 50-60 лет;  
-проживает в основном в Ярославле;  
-большинство этих клиентов не имеют кредитную карту;  
-высокая активность;  
-в среднем 4 и более объектов в собственности;  
-в среднем оформлено 4 продукта;  
-мужчин в 2 раза больше, чем женщин,  
-баланс на счете 100-400 т.р.,  
-баллы кредитного скоринга 825-900.

#### Выводы по сегментации отточных клиентов:
В результате выделения сегментов:  
-мужчины без кредитных карточек,  
-мужчины с 4-мя продуктами,  
-активные и имеющие > 4 объектов в собственности,  
-зрелого возраста с балансом 100-400 т.р.,  
-зрелого возраста с доходом 100-200 т.р.,  
-с баллами 800-900 и балансом на счете 100-400 т.р.,  
получили наибольшие доли отточности по категориям 'мужчины с 4-мя продуктами', 'мужчины без кредитных карт' и 'активные с более 4-мя объектами в собственности'.

##### Рекомендации по сегментам

Пожалуй, основной акцент имеет смысл сделать на клиентов мужского пола с 4-мя продуктами. Возможно нужно обратить внимание на изменения условий при переходе с 4-го продукта к 5-му, с 5-го к 6-му продукту и т.д, возможно условия ухудшаются.  

Также имеет смысл подумать над специальными предложениями для мужчин - их в группе оттока больше, чем в группе тех, кто остался. Можно разработать специальные предложения именно мужчинам без кредитных карт, например выпустить кредитки с хорошими условиями бонусов специально к праздничным дня, посвященным мужчинам.  

Мы помним, что большая часть отточных клиентов не имеют кредитных карт и можно предположить, что карта является стимулом для начала договорных отношений с банком. Возможно пересмотр системы бонусов, которая стимулировала бы ее использование и не позволял уходить пользователям из банка, решил бы в какой-то мере проблему оттока.  

Что касаемо возрастных клиентов с доходом 100-200 т.р., например, предложить открыть накопительные счета, вклады и кредитные программы для людей старше 50 лет, сделав акцент на их будущем, будущем их детей и внуков.